# Probe Designer


## Environment


In [1]:
# basci env
import os
from pathlib import Path
import sys
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# get gene data from ncbi
from Bio import Entrez

# blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append("../lib")

# dir
DATASET_DIR = Path('F:\spatial_data\dataset_probe')
RUNID = '20240808_Z.Y_Mouse'
workdir = DATASET_DIR / RUNID
os.makedirs(workdir, exist_ok=True)

In [2]:
# create results dir
current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)
output = os.path.join(workdir, 'results', formatted_time+'_NCBI')
bds_candidate_dir = os.path.join(output, "bds_candidate")
os.makedirs(output, exist_ok=True)
os.makedirs(bds_candidate_dir, exist_ok=True)

# file name variables
bds_candidate_file_suffix = "_bds_candidate.fasta"
combined_bds_candidates_file = "total_bds_candidate.fasta"
combined_bds_candidates_blast_file = "total_bds_candidate_blast.fasta"
bds_candidate_num_file = "bds_candidate_num.json"
blast_results_file = "blast_results.xml"

# tmp file
gene_name_list_file = "gene_name_list.txt"
gene_id_name_file = "gene_id_list.txt"
gene_seq_in_file = "gene_seq_in_file.gb"
blast_results_file = "blast_results.xml"

## Get genbank file of each gene from ncbi dataset

https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch


In [3]:
# Get gene id and other information from ncbi dataset(api)
## Generate gene_search_list from gene_name_list
organism_of_interest = "mouse"
n_type_of_interest = "mRNA"
with open(os.path.join(output, gene_name_list_file)) as f: gene_name_list = f.read().splitlines()  # Read name_list from existing file
with open(os.path.join(output, gene_id_name_file), "r") as f: id_list = f.read().split("\n")       # Read id_list from existing file

In [ ]:
# ## Get gene id list using Entrez.esearch
# gene_search_list = [", ".join([name, organism_of_interest, n_type_of_interest])
#     for name in gene_name_list]
# id_list = []
# for gene_search in gene_search_list:
#     Entrez.email = "1418767067@qq.com"
#     handle = Entrez.esearch(db="nuccore", term=gene_search)
#     record = Entrez.read(handle)
#     handle.close()
#     id_list += record["IdList"][:1]  # set number of search results to read
# with open(tmp + gene_id_name_file, "w") as f:
#     f.write("\n".join(id_list))

In [4]:
# Get the genbank file of each gene by id list
fetch_per_round = 3
round = -(-len(id_list) // fetch_per_round)

# initialization of gb file
with open(os.path.join(output, gene_seq_in_file), "w") as f: f.write("")

for i in tqdm(range(round)):
    id_list_per_round = id_list[i * fetch_per_round : (i + 1) * fetch_per_round]
    Entrez.email = "1418767067@qq.com"
    handle = Entrez.efetch(
        db="nuccore",
        strand=1,  # plus if strand=1
        id=id_list_per_round,
        rettype="gbwithparts",
        retmode="text",
    )
    seq_record = handle.read()
    handle.close()
    with open(os.path.join(output, gene_seq_in_file), "a") as f:
        f.write(seq_record)

100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


## Binding site Searcher


In [5]:
from lib.search_binding import position_search, optimize_subsequence, gb_extract

# Initiation of array
binding_site_entry = [
    "accession", "gene_name", "mol_type", "organism",
    "pos_on_seq", "binding", "Tm", "Tm_l", "Tm_r", "mfe", "wanted"]
align_entry = ["align_num", "align_accession", "align_descrip", "plus/minus"]
BDS_INFO = pd.DataFrame(columns=binding_site_entry + align_entry)

# Search binding sites on mRNA sequence
file_in = os.path.join(output, gene_seq_in_file)
file_out_dir = bds_candidate_dir
os.makedirs(file_out_dir, exist_ok=True)

pre_binding_num = {}

# initialization of file
with open(os.path.join(output, combined_bds_candidates_file), "w") as handle: handle.write("")
with open(os.path.join(output, combined_bds_candidates_blast_file), "w") as f: f.write("")
with open(os.path.join(output, gene_seq_in_file), 'r') as file_handle: gb_records = list(SeqIO.parse(file_handle, "genbank"))

for _, record in enumerate(tqdm(gb_records, position=0)):
    id, gene_name, mol_type, organism, minus_seq = gb_extract(record, gene_name=gene_name_list[_], CDS=True)
    pos_info = position_search(
        minus_seq, gene=gene_name,
        BDS_len=40, BDS_num=50, min_gap=1, better_gap=40,
        G_min=0.25, G_max=0.7, G_consecutive=5, Tm_low=50, Tm_high=65, 
        verbose_pos=1, leave=False, warn=False)
    
    record_list = []
    for i, seq in enumerate([_['seq'] for _ in pos_info]):
        record_list.append(
            SeqRecord(Seq(seq), id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type])))

    # add information about binding sites to FOI
    add = pd.DataFrame(pos_info)
    add['accession'] = id
    add['gene_name'] = gene_name
    add['mol_type'] = mol_type
    add['organism'] = organism
    BDS_INFO = pd.concat([BDS_INFO, add], ignore_index=True)

    file_out = os.path.join(file_out_dir, gene_name + bds_candidate_file_suffix)
    
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list: SeqIO.write(new_record, f, "fasta")
    with open(os.path.join(output, combined_bds_candidates_file), "a") as handle:
        for new_record in record_list: SeqIO.write(new_record, handle, "fasta")
    with open(os.path.join(output, combined_bds_candidates_blast_file), "a") as handle:
        for new_record in record_list: 
            seq = str(new_record.seq)
            seq = seq[len(seq)//2-16:len(seq)//2+16]
            new_record = SeqRecord(Seq(seq), id=new_record.id, description=new_record.description)
            SeqIO.write(new_record, handle, "fasta")
    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(pos_info)

with open(os.path.join(output, bds_candidate_num_file), "w") as f: json.dump(pre_binding_num, f)

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\Mingchuan\AppData\Local\Temp\ipykernel_17416\3035274352.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  BDS_INFO = pd.concat([BDS_INFO, add], ignore_index=True)
100%|██████████| 3/3 [00:00<00:00, 34.88it/s]


## Blast and extract blast results

NCBIXML: https://homolog.us/Biopython/Bio.Blast.NCBIXML.html#read/0

BlastRecord: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html

XMLReader: https://codebeautify.org/xmlviewer#


In [ ]:
# with open(file_out_dir + total_pre_binding_file_name, "r") as f:
#     fasta_string = f.read()
# txid = [2697049]  # organism

# # Submit BLAST search and get handle object
# handle = NCBIWWW.qblast(
#     program="blastn",
#     megablast="yes",
#     database="refseq_rna",
#     sequence=fasta_string,
#     url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
#     format_object="Alignment",
#     format_type="Xml",
# )

# # read handle object and save to a file
# with open(tmp + blast_results_file, "w") as f:
#     f.write(handle.read())

In [6]:
# Extract interested information from blast_results
align_num = []

# read the id/plus-minus part/align_num
with open(os.path.join(output, blast_results_file), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        BDS_INFO.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        BDS_INFO.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try: BDS_INFO.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except: BDS_INFO.loc[loca, "plus/minus"] = "NAN"

        loca += 1

BDS_INFO["align_num"] = align_num

## Select wanted binding site


In [7]:
# sieve for the suitable binding site
BDS_INFO["wanted"] = [True] * len(BDS_INFO)
verbose = False
gene_name_list = [_.upper() for _ in gene_name_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(BDS_INFO)):
    # check gene_name
    gene_name = BDS_INFO.loc[i, "gene_name"]
    if gene_name.upper() not in gene_name_list:
        BDS_INFO.loc[i, "wanted"] = False
        if verbose: print(f"Gene name: {gene_name} not in gene list.")
    else:
        try: gene_name_list_out.remove(gene_name)
        except: pass

    # check DNA or mRNA type
    if BDS_INFO.loc[i, "wanted"] == True:
        if BDS_INFO.loc[i, "mol_type"] != "mRNA":
            BDS_INFO.loc[i, "wanted"] = False
            if verbose: print(f"Gene name: {gene_name} mRNA.")

    # check gene_organism name
    if BDS_INFO.loc[i, "wanted"] == True:
        spe_ori, gene_ori = BDS_INFO.loc[i, "organism"], BDS_INFO.loc[i, "gene_name"]
        descrip = BDS_INFO.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                BDS_INFO.loc[i, "wanted"] = False
                if verbose: print(f"Gene name: {gene_ori} not specific.")
                break

    # check plus/minus
    if BDS_INFO.loc[i, "wanted"] == True:
        if pd.isnull(BDS_INFO.loc[i, "plus/minus"]) or BDS_INFO.loc[i, "plus/minus"]=='':
            BDS_INFO.loc[i, "wanted"] = False
            if verbose: print(f"Gene name: {gene_ori} not found in BLAST.")
        else:
            pm_list = BDS_INFO.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                BDS_INFO.loc[i, "wanted"] = False
                if verbose: print(f"Gene name: {gene_ori} not plus/minus.")

# write the whole information of interest to a excel file in tmp dir
BDS_INFO.to_excel(os.path.join(output, "probes_candidates.xlsx"))

out_tmp = BDS_INFO[BDS_INFO["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_wanted = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    pos_best = optimize_subsequence(pos_wanted, length=3, min_gap=40, better_gap=80, gene=gene)
    pos_output = out_tmp[out_tmp.gene_name == gene]
    pos_output = pos_output[pos_output["pos_on_seq"].isin(pos_best)]
    output_df = pd.concat([output_df, pos_output])

# write the output to a xlsx file
output_df.to_excel(os.path.join(output, "probes_wanted.xlsx"))

Gene Myl2: Not enough pos for 3 binding sites.
Gene Myl2: condition too harsh, loose to get better results
[107, 108, 111, 116, 140, 141, 245, 246, 247, 248, 249, 250, 251, 252]
